### 데이터 전처리 및 모델학습

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
import json

In [3]:
# JSON 파일에서 데이터 로드
with open('users_data.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# DataFrame으로 변환
df = pd.DataFrame(data)
df

,fields
0,"{'username': 'UlkQe73', 'last_name': '우', 'fir..."
1,"{'username': 'QazDe31', 'last_name': '람', 'fir..."
2,"{'username': 'NodLl77', 'last_name': '우', 'fir..."
3,"{'username': 'TuvEf10', 'last_name': '은', 'fir..."
4,"{'username': 'JuzGi61', 'last_name': '후', 'fir..."
...,...
9995,"{'username': 'NuzYy16', 'last_name': '호', 'fir..."
9996,"{'username': 'XobUw49', 'last_name': '연', 'fir..."
9997,"{'username': 'DizXv49', 'last_name': '영', 'fir..."
9998,"{'username': 'KepCe6', 'last_name': '서', 'firs..."


In [4]:
# 'fields' 열에서 사용자 정보 추출하여 DataFrame 확장
df = pd.concat([df.drop(['fields'], axis=1), df['fields'].apply(pd.Series)], axis=1)

# 데이터 출력
df.head()

,username,last_name,first_name,nickname,financial_products,age,currentAsset,salary,password,gender
0,UlkQe73,우,정인,건이비비,"10527001001272000,10527001001272000,TD11330031...",85,32800000,556000000,1234,0
1,QazDe31,람,이장,하람나무,"10527001000925000,10-01-30-031-0048-0000,10-04...",83,19900000,698000000,1234,0
2,NodLl77,우,윤장,새봄마일,"101272000006,10527001001272000,TD11300031000,0...",59,33000000,765000000,1234,1
3,TuvEf10,은,황민,봄이하랑,01020400530001,32,66100000,333000000,1234,0
4,JuzGi61,후,정지,바름반디,"10141114300011,230-0119-85,10-003-1225-0001",19,16300000,851000000,1234,1


In [5]:
# financial_products 열의 데이터를 분리하여 각 상품을 별도의 행으로 확장
df = df.assign(financial_products=df['financial_products'].str.split(',')).explode('financial_products')

# 비어 있는 financial_products 행 삭제
df = df[df['financial_products'].str.strip().astype(bool)]

# 인덱스 재설정
df.reset_index(drop=True, inplace=True)

df


,username,last_name,first_name,nickname,financial_products,age,currentAsset,salary,password,gender
0,UlkQe73,우,정인,건이비비,10527001001272000,85,32800000,556000000,1234,0
1,UlkQe73,우,정인,건이비비,10527001001272000,85,32800000,556000000,1234,0
2,UlkQe73,우,정인,건이비비,TD11330031000,85,32800000,556000000,1234,0
3,UlkQe73,우,정인,건이비비,01030500510002,85,32800000,556000000,1234,0
4,UlkQe73,우,정인,건이비비,10141109800021,85,32800000,556000000,1234,0
...,...,...,...,...,...,...,...,...,...,...
25078,DizXv49,영,임도,하람루리,10120114700011,40,8000000,598000000,1234,1
25079,DizXv49,영,임도,하람루리,01013000110000000001,40,8000000,598000000,1234,1
25080,KepCe6,서,조현,시우온유,21001115,31,81300000,1000000000,1234,1
25081,KepCe6,서,조현,시우온유,230-0118-14,31,81300000,1000000000,1234,1


In [6]:
# # financial_products 열의 데이터를 원-핫 인코딩
# encoded_df = pd.get_dummies(df, columns=['financial_products'])

# # 원-핫 인코딩을 적용한 DataFrame에서 종속 변수와 독립 변수 분리
# X = encoded_df[['age', 'currentAsset', 'salary','gender']]
# y = encoded_df.drop(['pk', 'username', 'last_name', 'first_name', 'nickname', 'age', 'currentAsset', 'salary', 'password', 'gender'], axis=1)
# # 훈련 세트와 테스트 세트로 분할
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# from sklearn.ensemble import RandomForestClassifier
# # 각 상품을 예측하는 이진 분류 모델을 생성하고 학습
# models = {}
# for column in y.columns:
#     model = RandomForestClassifier(n_estimators=100, random_state=42)
#     model.fit(X_train, y_train[column])
#     models[column] = model

# # 각 모델을 사용하여 예측
# predictions = {}
# for column, model in models.items():
#     predictions[column] = model.predict(X_test[column])

# # predictions 딕셔너리에 각 상품에 대한 예측 결과가 저장됩니다.


In [7]:
from sklearn.preprocessing import LabelEncoder

# LabelEncoder를 사용하여 financial_products 열의 값을 정수로 인코딩
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(df['financial_products'])

# 인코딩된 값을 DataFrame에 할당
df['financial_products_encoded'] = encoded_labels

# 독립 변수와 종속 변수 분리
X = df[['age', 'currentAsset', 'salary','gender']]
y = df['financial_products_encoded']

# Train-Test 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 다중 분류 모델 정의 및 학습
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [8]:
from sklearn.metrics import accuracy_score, classification_report

# 테스트 데이터에 대한 예측
y_pred = model.predict(X_test)

# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# 분류 보고서 출력
print(classification_report(y_test, y_pred))


Accuracy: 0.008570859079130955
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        48
           1       0.02      0.02      0.02        57
           2       0.02      0.02      0.02        55
           3       0.00      0.00      0.00        37
           4       0.00      0.00      0.00        47
           5       0.00      0.00      0.00        46
           6       0.00      0.00      0.00        54
           7       0.04      0.03      0.04        60
           8       0.00      0.00      0.00        43
           9       0.00      0.00      0.00        43
          10       0.00      0.00      0.00        56
          11       0.00      0.00      0.00        43
          12       0.00      0.00      0.00        54
          13       0.02      0.02      0.02        62
          14       0.00      0.00      0.00        46
          15       0.02      0.02      0.02        52
          16       0.06      0.04      0.05       

### 위에서 보다시피... 랜덤 데이터라 정확성 난리남

In [9]:
# 예측값을 다시 실제 상품으로 변환
predicted_products = label_encoder.inverse_transform(y_pred)

# 예측값 출력
print("Predicted Products:", predicted_products)

Predicted Products: ['101272000058' '10-01-30-031-0036' '10511008001004000' ...
 '10511008001278000' '21001236' '010200100084']


### 예측 테스트 해보기

In [10]:
# 주어진 특성을 변수에 저장
new_data = {
    'age': 10,
    'currentAsset': 2600000,
    'salary': 154600000,
    'gender': 0
}

# 입력 데이터를 DataFrame으로 변환
new_df = pd.DataFrame([new_data])

# 모델을 사용하여 예측
predicted_label = model.predict(new_df)

# 예측된 정수 값을 실제 상품으로 변환
predicted_product = label_encoder.inverse_transform(predicted_label)

# 예측 결과 출력
print("Predicted Product:", predicted_product[0])


Predicted Product: TD11300031000


### 모델 저장하기

In [ ]:
import joblib

# 모델 저장
joblib.dump(model, 'random_forest_model.pkl')

In [ ]:
# 라벨 인코더를 파일에 저장
joblib.dump(label_encoder, 'label_encoder.pkl')



In [ ]:
# import joblib

# # 모델 불러오기
# loaded_model = joblib.load('random_forest_model.pkl')

# # 저장된 라벨 인코더를 로드
# loaded_label_encoder = joblib.load('label_encoder.pkl')

# # 불러온 모델과 인코더를 이용하여 예측 등의 작업 수행

랜덤포레스트 이외에도 로지스틱, 결정트리, SVM, kNN, 나이브베이즈, 신경망 등으로 이용할수 있겠지만... 할까마까 고민

### 모델, 인코더 불러와서 적용해보기

In [10]:
import pandas as pd
import joblib

In [11]:
# 모델 불러오기
loaded_model = joblib.load('random_forest_model.pkl')

# 저장된 라벨 인코더를 로드
loaded_label_encoder = joblib.load('label_encoder.pkl')

In [12]:
# 예시 데이터
new_data = {
    'age': 10,
    'currentAsset': 2600000,
    'salary': 154600000,
    'gender': 0
}
new_df = pd.DataFrame([new_data])

In [13]:
# 모델을 사용하여 예측 확률 얻기
predicted_probabilities = loaded_model.predict_proba(new_df)

# 예측된 확률과 레이블을 함께 저장할 리스트 생성
predicted_results = []

# 모든 레이블에 대해 예측 확률을 반복하여 리스트에 저장
for label, prob in zip(loaded_model.classes_, predicted_probabilities[0]):
    predicted_results.append((label, prob))

# 확률이 높은 순으로 정렬
predicted_results.sort(key=lambda x: x[1], reverse=True)

# 상위 5개 예측 출력
top_n_predictions = 5
for i in range(top_n_predictions):
    label, prob = predicted_results[i]
    predicted_product = loaded_label_encoder.inverse_transform([label])[0]
    print(f"Top {i+1} Predicted Product: {predicted_product}, Probability: {prob:.4f}")

Top 1 Predicted Product: 200-0135-12, Probability: 0.2500
Top 2 Predicted Product: 10141109800021, Probability: 0.0907
Top 3 Predicted Product: TD11330029000, Probability: 0.0700
Top 4 Predicted Product: 21001203, Probability: 0.0647
Top 5 Predicted Product: 010300100335, Probability: 0.0600
